In [1]:
import flash as fz
import joblib
import klib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import seaborn as sns
import yaml
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.ensemble import (
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    VotingClassifier,
)
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    LabelEncoder,
    MinMaxScaler,
    OneHotEncoder,
    RobustScaler,
    StandardScaler,
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# from xgboost import XGBClassifier

# Data Assessment and Cleaning

In [2]:
df = pd.read_csv("data/processed/train.csv")

In [3]:
df

,gender,married,dependents,education,self_employed,applicant_income,coapplicant_income,loan_amount,loan_amount_term,credit_history,property_area,loan_status
0,male,yes,0,graduate,no,2333,2417.0,136.0,360.0,1.0,urban,yes
1,female,yes,1,graduate,yes,11500,0.0,286.0,360.0,0.0,urban,no
2,female,no,0,graduate,no,645,3683.0,113.0,480.0,1.0,rural,yes
3,male,yes,0,graduate,no,4750,2333.0,130.0,360.0,1.0,urban,yes
4,male,yes,2,not_graduate,no,1993,1625.0,113.0,180.0,1.0,semiurban,yes
...,...,...,...,...,...,...,...,...,...,...,...,...
486,male,yes,2,graduate,no,5000,3667.0,236.0,360.0,1.0,semiurban,yes
487,male,yes,0,graduate,no,9083,0.0,228.0,360.0,1.0,semiurban,yes
488,male,yes,0,not_graduate,NaN,2894,2792.0,155.0,360.0,1.0,rural,yes
489,male,yes,0,graduate,no,2130,6666.0,70.0,180.0,1.0,semiurban,no


In [5]:
df['education'].unique()

array(['graduate', 'not graduate'], dtype=object)

In [3]:
pd.read_csv("data/processed/test.csv")

,gender,married,dependents,education,self_employed,applicant_income,coapplicant_income,loan_amount,loan_amount_term,credit_history,property_area,loan_status
0,NaN,yes,3.0,graduate,no,23803,0.0,370.0,360.0,1.0,rural,y
1,male,yes,0.0,graduate,no,7933,0.0,275.0,360.0,1.0,urban,n
2,male,yes,0.0,not graduate,yes,2609,3449.0,165.0,180.0,0.0,rural,n
3,male,yes,2.0,not graduate,no,2889,0.0,45.0,180.0,0.0,urban,n
4,male,yes,3.0,graduate,no,4000,7750.0,290.0,360.0,1.0,semiurban,n
...,...,...,...,...,...,...,...,...,...,...,...,...
118,male,yes,0.0,graduate,no,8334,0.0,160.0,360.0,1.0,semiurban,n
119,male,yes,0.0,graduate,no,3173,3021.0,137.0,360.0,1.0,urban,y
120,male,yes,0.0,not graduate,no,2600,1911.0,116.0,360.0,0.0,semiurban,n
121,male,yes,2.0,graduate,no,5819,5000.0,120.0,360.0,1.0,rural,y


In [ ]:
# Load the dataset
df = pd.read_csv("data/raw/loan_sanction_train.csv")

In [7]:
df["dependents"].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
# Understand structure of the dataset
df.head()

In [ ]:
# Drop useless features
useless_features = ["Loan_ID"]
df.drop(columns=useless_features, inplace=True)

# Test
df.columns

In [ ]:
# Clean column names
df = klib.clean_column_names(df)

# Test
column_names = df.columns.tolist()
column_names

In [ ]:
# Check for duplicate data points
def check_duplicates(df):
    if df.duplicated().any():
        return df[df.duplicated(keep=False)]
    return "There are no duplicate data points in the dataset."


check_duplicates(df)

In [ ]:
# Get some information about the dataset
df.info()

In [ ]:
# Extract numerical, categorical, and other features from the dataset
num_cols, cat_cols, other_cols = fz.extract_features(
    df, "all", ignore_cols=["loan_status"]
)

In [ ]:
# Print numerical features of dataset
df[num_cols]

In [ ]:
# Print categorical features of dataset
df[cat_cols]

In [ ]:
df["dependents"] = df["dependents"].str.replace(r"\+$", "", regex=True)

In [ ]:
df["loan_status"] = df["loan_status"].replace({"Y": "yes", "N": "no"})

In [ ]:
# Convert datatype of features
# df['applicant_income'] = df['applicant_income'].astype(float)
df["dependents"] = df["dependents"].astype("Int64")
df["loan_amount_term"] = df["loan_amount_term"].astype("Int64")
df["credit_history"] = df["credit_history"].astype("Int64")
# Convert categorical features' data type to category
# This will be helpful while doing analysis
df[cat_cols] = df[cat_cols].astype("category")

# Test
df.dtypes

In [ ]:
# Numerical features
num_nan_pct = fz.calc_nan_values(df[num_cols])
num_cols_with_nan = num_nan_pct.index.tolist()

print(num_nan_pct)  # Percentage of missing values in numerical features
print(num_cols_with_nan)  # Numerical features with missing values

In [ ]:
# Categorical features
cat_nan_pct = fz.calc_nan_values(df[cat_cols])
cat_cols_with_nan = cat_nan_pct.index.tolist()

print(cat_nan_pct)  # Percentage of missing values in categorical features
print(cat_cols_with_nan)  # Categorical features with missing values

In [ ]:
# Check whether the target column contains any missing values
df["loan_status"].isna().sum()

In [ ]:
# Visualize the distribution of missing values to determine the type of missing values
fig, axs = fz.nan_value_viz(df[num_cols_with_nan + cat_cols_with_nan])
fig

# EDA

## Univariate analysis

### Features

#### Numerical

In [ ]:
# Statistical measures
df[num_cols].describe().T

In [ ]:
# Statistical moments
fz.stats_moments(df[num_cols])

In [ ]:
# Plotting histogram & boxplot
fig, axs = fz.hist_box_viz(df[num_cols])
fig

#### Categorical

In [ ]:
# Statistical measures
df[cat_cols].describe().T

In [ ]:
# Countplots
fig, axs = fz.count_viz(df[cat_cols])
fig

### Target

In [ ]:
df["loan_status"].describe().T

In [ ]:
plt.pie(
    df["loan_status"].value_counts(),
    labels=df["loan_status"].unique(),
    autopct="%0.2f%%",
    shadow=True,
    explode=(0, 0.1),
    counterclock=False,
    colors=["lime", "cyan"],
)
plt.show()

## Bivariate analysis

### Feature vs. Feature

#### Numerical - Numerical

In [ ]:
# Pairplot
grid = fz.pair_viz(df[num_cols])
plt.show()

In [ ]:
# Correlation heatmap
methods = ["pearson", "spearman", "kendall"]
for method in methods:
    fz.corr_heatmap_viz(df[num_cols], method=method)
    plt.show()
    print("-" * 150)

#### Categorical - Categorical

In [ ]:
# Crosstab Heatmap
fz.crosstab_heatmap_viz(df, cat_cols, normalize="both")

#### Numerical - Categorical

##### Box-plot

In [ ]:
# applicant_income
fig, axs = fz.num_cat_viz(df, "applicant_income", cat_cols)
fig

In [ ]:
# coapplicant_income
fig, axs = fz.num_cat_viz(df, "coapplicant_income", cat_cols)
fig

In [ ]:
# loan_amount
fig, axs = fz.num_cat_viz(df, "loan_amount", cat_cols)
fig

##### KDE-plot

In [ ]:
# applicant_income
fig, axs = fz.num_cat_viz(df, "applicant_income", cat_cols, kind="kde")
fig

In [ ]:
# coapplicant_income
fig, axs = fz.num_cat_viz(df, "coapplicant_income", cat_cols, kind="kde")
fig

In [ ]:
# loan_amount
fig, axs = fz.num_cat_viz(df, "loan_amount", cat_cols, kind="kde")
fig

##### Point-plot

In [ ]:
# applicant_income
fig, axs = fz.num_cat_viz(df, "applicant_income", cat_cols, kind="point")
fig

In [ ]:
# coapplicant_income
fig, axs = fz.num_cat_viz(df, "coapplicant_income", cat_cols, kind="point")
fig

In [ ]:
# loan_amount
fig, axs = fz.num_cat_viz(df, "loan_amount", cat_cols, kind="point")
fig

### Feature vs. Target

#### Categorical - Categorical

In [ ]:
# Crosstab Heatmap
fz.crosstab_heatmap_viz(df, cat_cols, ["loan_status"], "both")

#### Numerical - Categorical

In [ ]:
# Box-plot
fig, axs = fz.num_cat_viz(df, num_cols, "loan_status")
fig

In [ ]:
# KDE-plot
fig, axs = fz.num_cat_viz(df, num_cols, "loan_status", kind="kde")
fig

In [ ]:
# Point-plot
fig, axs = fz.num_cat_viz(df, num_cols, "loan_status", kind="point")
fig

## Mulitvariate analysis

### Feature vs. Feature

#### Numerical - Numerical - Numerical

In [ ]:
trace = go.Scatter3d(
    x=df["applicant_income"],
    y=df["coapplicant_income"],
    z=df["loan_amount"],
    mode="markers",
    marker=dict(size=5),
)

layout = go.Layout(
    scene=dict(
        xaxis_title="Applicant Income",
        yaxis_title="Coapplicant Income",
        zaxis_title="Loan Amount",
    )
)

fig = go.Figure(data=[trace], layout=layout)
fig.show()

#### Numerical - Numerical - Categorical

In [ ]:
# Pairplot
for feature in cat_cols:
    sns.pairplot(df, vars=num_cols, hue=feature)
    plt.show()
    print("-" * 105)

In [ ]:
def num_num_cat_viz(x, y, categorical_features):
    for feature in categorical_features:
        sns.relplot(df, x=x, y=y, col=feature)
        plt.show()
        print("-" * 118)

In [ ]:
# applicant_income & coapplicant_income
num_num_cat_viz("applicant_income", "coapplicant_income", cat_cols)

In [ ]:
# applicant_income & loan_amount
num_num_cat_viz("applicant_income", "loan_amount", cat_cols)

In [ ]:
# coapplicant_income & loan_amount
num_num_cat_viz("coapplicant_income", "loan_amount", cat_cols)

#### Numerical - Categorical - Categorical

### Feature vs. Target

#### Numerical - Numerical - Categorical

In [ ]:
sns.pairplot(df, vars=num_cols, hue="loan_status")

In [ ]:
def relplot(df, numerical_features, categorical_feature):
    for i, feature_i in enumerate(numerical_features):
        for j, feature_j in enumerate(numerical_features[i + 1 :], start=i + 1):
            sns.relplot(df, x=feature_i, y=feature_j, col=categorical_feature)
            plt.show()
            print("-" * df[categorical_feature].nunique() * 59)

In [ ]:
relplot(df, num_cols, "loan_status")

#### Numerical - Categorical - Categorical

# Feature construction

- It looks like people with a co-applicant income of 0 doesn't have a co-applicant. So, we should create a new feature called 'has_coapplicant'. For this feature, set the value to 'no' for individuals with a co-applicant income of 0, and 'yes' for those with a non-zero co-applicant income.

In [ ]:
df["has_coapplicant"] = np.where(df["coapplicant_income"] == 0, "no", "yes")

# Test
df["has_coapplicant"]

In [ ]:
# Appending newly created features based on their feature type
cat_cols.append("has_coapplicant")

# Test
cat_cols

# Feature transformation

In [ ]:
transformed_data = fz.feature_transform(df[num_cols])

## applicant_income

In [ ]:
# Histogram
fig, axs = fz.feature_transform_viz(df["applicant_income"], transformed_data)
fig

In [ ]:
# QQ Plot
fig, axs = fz.feature_transform_viz(df["applicant_income"], transformed_data, kind="qq")
fig

## coapplicant_income

In [ ]:
# Histogram
fig, axs = fz.feature_transform_viz(df["coapplicant_income"], transformed_data)
fig

In [ ]:
# QQ Plot
fig, axs = fz.feature_transform_viz(
    df["coapplicant_income"], transformed_data, kind="qq"
)
fig

## loan_amount

In [ ]:
# Histogram
fig, axs = fz.feature_transform_viz(df["loan_amount"], transformed_data)
fig

In [ ]:
# QQ Plot
fig, axs = fz.feature_transform_viz(df["loan_amount"], transformed_data, kind="qq")
fig

## Conclusions

- **applicant_income & loan_amount:** Quantile Transform normalizes the data effectively.  
- **coapplicant_income:** Reciprocal Transform transforms coapplicant_income to follow a bimodal distribution.

In [ ]:
df["applicant_income"] = transformed_data["Quantile"]["applicant_income"]
df["coapplicant_income"] = transformed_data["Reciprocal"]["coapplicant_income"]
df["loan_amount"] = transformed_data["Quantile"]["loan_amount"]

In [ ]:
# Test
fig, axs = fz.hist_box_viz(df[num_cols])
fig

## Handle Missing Values

### Numerical

In [ ]:
# 1. Handle missing values in categorical columns
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [ ]:
# OneHotEncoder expects datatypes of every value in a column to be the same
X[cat_cols] = X[cat_cols].astype(str)

In [ ]:
# Store encoded feature names before encoding
unique_values_in_cols = {}
for col in cat_cols:
    encoded_columns = []
    for value in X[col].unique():
        encoded_columns.append(f"{col}_{value}")
    unique_values_in_cols[col] = encoded_columns

In [ ]:
# 2. One-Hot Encode categorical features
encoder = OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore")
encoded_X_data = encoder.fit_transform(X[cat_cols])
encoded_X_df = pd.DataFrame(encoded_X_data, columns=encoder.get_feature_names_out())

# Concatenating encoded categorical features with the rest of the X
X = pd.concat([X.drop(columns=cat_cols), encoded_X_df], axis=1)

In [ ]:
# 3. Impute missing values in numerical features using KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)
X = pd.DataFrame(knn_imputer.fit_transform(X), columns=X.columns)

In [ ]:
# 4. Assign the missing value imputed numerical features back to df
df[num_cols_with_nan] = X[num_cols_with_nan]

### Categorical

In [ ]:
def advanced_categorical_imputer(X, y, clf_model):
    X, y = X.copy(), y.copy()  # Avoid modifying the original X and y

    y_notna = y.notna()  # Create a mask for non-missing values in y

    # Split the data into training (non-missing) and test (missing) data
    X, X_test = X[y_notna], X[~y_notna]
    y_train, y_test = y[y_notna], y[~y_notna]

    # Label encoding the target feature
    le = LabelEncoder()
    y_train = le.fit_transform(y_train)

    clf_model.fit(X, y_train)  # Train the model

    y_pred = clf_model.predict(X_test)  # Predict on the test data (missing values)

    # Inverse transform the predicted values to original labels
    y_pred_inverse = le.inverse_transform(y_pred)

    y[y_test.index] = y_pred_inverse  # Impute the missing target values

    return y, clf_model, le

In [ ]:
clf_models = {}
label_encoders = {}
for col in cat_cols_with_nan:
    df[col], clf_models[col], label_encoders[col] = advanced_categorical_imputer(
        X.drop(columns=unique_values_in_cols[col], errors="ignore"),
        df[col],
        ExtraTreesClassifier(random_state=42),
    )

In [ ]:
def check_missing_values(df):
    if df.isna().any().any():
        print("There are still missing values in the DataFrame.")
    else:
        print("There are no missing values left in the DataFrame.")


# Test
check_missing_values(df)

In [ ]:
# Load the configurations
with open("config/config_v2.toml", "r") as file:
    config_data = toml.load(file)

num_cols, cat_cols = config_data["num"]["cols"], config_data["cat"]["cols"]

# Model Building

In [ ]:
# Split the data into features and target
X = df.drop("loan_status", axis=1)
y = df["loan_status"]

In [ ]:
# Label encode target
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
# Tranformer for preprocessing data
transformer = make_column_transformer(
    (StandardScaler(), num_cols),
    (OneHotEncoder(drop="first", sparse_output=False), cat_cols),
    remainder="passthrough",
)

In [ ]:
X_transformed = transformer.fit_transform(X)

# Test
X_transformed.shape

## Handling imbalanced dataset

In [ ]:
# Oversampling the dataset using SMOTE
smote = SMOTE(random_state=42)
X_transformed, y_resampled = smote.fit_resample(X_transformed, y)

# Test
print(X_transformed.shape, y_resampled.shape)

In [ ]:
# Test
unique_values, counts = np.unique(y_resampled, return_counts=True)

# Print the counts of each class
for value, count in zip(unique_values, counts):
    print(f"Class {value}: {count}")

## Model selection (Before hyperparameter tuning)

In [ ]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Support Vector Machine": SVC(),
    "KNN": KNeighborsClassifier(),
    "Decision Trees": DecisionTreeClassifier(),
    "Xgboost": XGBClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
}

In [ ]:
# Define metric functions
metrics = {
    "accuracy": accuracy_score,
    "precision": precision_score,
    "recall": recall_score,
    "f1": f1_score,
}

In [ ]:
def eval_models_across_metrics(models, metrics, X, y, cv=5):
    models_across_metrics = {metric: {} for metric in metrics}
    for metric in metrics:
        for model_name, model in models.items():
            cv_scores = cross_val_score(model, X, y, cv=cv, scoring=metric)
            cv_scores_mean = cv_scores.mean()
            models_across_metrics[metric][model_name] = round(cv_scores_mean, 3)
    return pd.DataFrame(models_across_metrics)

In [ ]:
models_across_metrics = eval_models_across_metrics(
    models, metrics.keys(), X_transformed, y_resampled
)

In [ ]:
models_across_metrics

Conclusions:

- After evaluating the metrics, I have decided to focus on the top 3 models (in terms of accuracy_score): Random Forest Classifier, Extra Trees Classifier, Xgboost Classifier.

## Hyperparameter tuning

In [ ]:
# Define top models for further hyperparameter tuning
models = {
    "Random Forest": RandomForestClassifier(),
    "Xgboost": XGBClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
}

In [ ]:
# Split the dataset into features and target
X = df.drop("loan_status", axis=1)
y = df["loan_status"]

In [ ]:
# Define parameter grids
param_grids = {
    "Random Forest": {
        "n_estimators": [50, 200],
        "max_depth": [None, 30],
        "min_samples_split": [2, 10],
        "min_samples_leaf": [1, 4],
    },
    "Xgboost": {
        "n_estimators": [50, 200],
        "max_depth": [3, 10],
        "learning_rate": [0.01, 0.2],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0],
        "gamma": [0, 0.2],
    },
    "Extra Trees": {
        "n_estimators": [50, 200],
        "max_depth": [None, 30],
        "min_samples_split": [2, 10],
        "min_samples_leaf": [1, 4],
        "bootstrap": [True, False],
    },
}

In [ ]:
def perform_grid_search(models, param_grids, X, y):
    best_params = {}
    for model_name, model in models.items():
        print(f"Processing {model_name}...")
        param_grid = param_grids[model_name]
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring="accuracy",
            cv=5,
            n_jobs=-1,
            verbose=1,
        )
        grid_search.fit(X, y)
        best_params[model_name] = {
            "Best Parameters": grid_search.best_params_,
            "Average accuracy score on the best parameters": round(
                grid_search.best_score_, 3
            ),
        }
    return best_params

In [ ]:
# Finding best hyperparameters on top models using GridSearchCV
best_params = perform_grid_search(models, param_grids, X_transformed, y_resampled)

In [ ]:
pd.DataFrame(best_params)

In [ ]:
# best_params

# {'Random Forest': {'Best Parameters': {'max_depth': None,
#    'min_samples_leaf': 1,
#    'min_samples_split': 2,
#    'n_estimators': 200},
#   'Average accuracy score on the best parameters': 0.838},
#  'Xgboost': {'Best Parameters': {'colsample_bytree': 0.8,
#    'gamma': 0.2,
#    'learning_rate': 0.2,
#    'max_depth': 10,
#    'n_estimators': 50,
#    'subsample': 1.0},
#   'Average accuracy score on the best parameters': 0.826},
#  'Extra Trees': {'Best Parameters': {'bootstrap': True,
#    'max_depth': 30,
#    'min_samples_leaf': 1,
#    'min_samples_split': 2,
#    'n_estimators': 200},
#   'Average accuracy score on the best parameters': 0.84}}

In [ ]:
# Define top models with best hyperparameters
models = {
    "Random Forest": RandomForestClassifier(
        **best_params["Random Forest"]["Best Parameters"]
    ),
    "Xgboost": XGBClassifier(**best_params["Xgboost"]["Best Parameters"]),
    "Extra Trees": ExtraTreesClassifier(
        **best_params["Extra Trees"]["Best Parameters"]
    ),
}

In [ ]:
# Comparing top models across metrics after hyperparameter tuning
models_across_metrics = eval_models_across_metrics(
    models, metrics.keys(), X_transformed, y_resampled
)

In [ ]:
models_across_metrics


## Model training


In [ ]:
estimators = []
for model_name, model in models.items():
    estimators.append((model_name, model))

In [ ]:
estimators

In [ ]:
def eval_voting_clf(estimators, X, y, cv=5):
    # Create a voting classifier (hard voting)
    voting_clf_hard = VotingClassifier(estimators=estimators, voting="hard")

    # Create a voting classifier (soft voting)
    voting_clf_soft = VotingClassifier(estimators=estimators, voting="soft")

    # Apply cross-validation
    cv_scores_h = cross_val_score(voting_clf_hard, X, y, cv=cv, scoring="accuracy")
    cv_scores_s = cross_val_score(voting_clf_soft, X, y, cv=cv, scoring="accuracy")

    accuracy_results = {}

    accuracy_results["Hard Margin"] = round(cv_scores_h.mean(), 3)
    accuracy_results["Soft Margin"] = round(cv_scores_s.mean(), 3)

    return accuracy_results

In [ ]:
# Accuracy on hard and soft margin voting classifiers
accuracy = eval_voting_clf(estimators, X_transformed, y_resampled)
accuracy

In [ ]:
# Fit the best model
voting_clf = VotingClassifier(estimators, voting="hard")
voting_clf.fit(X_transformed, y_resampled)

In [ ]:
# Create a pipeline
pipeline = make_pipeline(
    StandardScaler(),  # Step 1: Standardize the data
    PCA(n_components=2),  # Step 2: Apply PCA
    LogisticRegression(),  # Step 3: Train a logistic regression model
)

In [ ]:
# Reorder columns
target_col = "loan_status"
df = df[num_cols + cat_cols + [target_col]]

# Test
df.columns

In [ ]:
# Pipeline
pipe = make_pipeline((transformer), ("model", voting_clf))

pipe.fit(X, y)

## Saving

In [ ]:
# Save the Machine Learning model
joblib.dump(voting_clf, "model.joblib")

In [ ]:
# Statistical measures
df[["has_coapplicant"]].describe().T

# Countplot
sns.countplot(x=df["has_coapplicant"])
plt.show()

# Heatmap
fz.crosstab_heatmap_viz(df, ["loan_status"], ["has_coapplicant"], "both")

# Heatmap
fz.crosstab_heatmap_viz(df, cat_cols, ["has_coapplicant"], "both")

# Point plot
fig, axs = fz.num_cat_viz(df, num_cols, "has_coapplicant", kind="point")
fig

In [ ]:
# Export config data
config_data = {
    "column_names": column_names,
    "num": {"cols": num_cols, "nan": num_cols_with_nan},
    "cat": {"cols": cat_cols, "nan": cat_cols_with_nan},
    "target_col": target_col,
}

with open("src/loan_sanction_prediction/config.yaml", "w") as file:
    yaml.dump(config_data, file, default_flow_style=False)

# Export dataset
fz.export(df, "../data/interim/cleaned_train_data_v1.csv", force_overwrite=True)

In [ ]:
log.info("Format columns...")
try:
    df = df[num_cols + cat_cols + target_col]
    log.success(f"Successfully formatted columns: {df.columns.to_list()}")
except Exception:
    log.exception("Failed to format columns")
    raise